In [28]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import ipywidgets as widgets
from IPython.display import display, clear_output

Initializing SIR_model and Gillespie's Algorithm functions

In [29]:
# Deterministic SIR model
def SIR_model(y, t, beta, gamma):
    S, I, R = y
    dS_dt = -beta * S * I / (S + I + R)
    dI_dt = beta * S * I / (S + I + R) - gamma * I
    dR_dt = gamma * I
    return [dS_dt, dI_dt, dR_dt]

def SIR_demography(y, t, beta, gamma, Lambda, mu):
    S, I, R = y
    dS = Lambda - beta * S * I - mu * S
    dI = beta * S * I - gamma * I - mu * I
    dR = gamma * I - mu * R
    return [dS, dI, dR]

def gillespie_algorithm(S0, I0, R0, beta, gamma, max_time):
    # Initial conditions
    S, I, R = S0, I0, R0
    t = 0
    times = [t]
    S_values = [S]
    I_values = [I]
    R_values = [R]

    while t < max_time and I > 0:
        N = S + I + R

        # Calculate propensities
        a1 = beta * S * I / N
        a2 = gamma * I
        a0 = a1 + a2

        # Time until next event
        dt = -np.log(np.random.random()) / a0
        t += dt

        # Determine which event occurs
        r = np.random.random()
        if r < a1 / a0:
            # Transmission event
            S -= 1
            I += 1
        else:
            # Recovery event
            I -= 1
            R += 1

        # Store results
        times.append(t)
        S_values.append(S)
        I_values.append(I)
        R_values.append(R)

    return times, S_values, I_values, R_values

def GA_with_demography(S0, I0, R0, beta, gamma, Lambda, mu, max_time):
    # Initial conditions
    S, I, R = S0, I0, R0
    t = 0
    times = [t]
    S_values = [S]
    I_values = [I]
    R_values = [R]

    while t < max_time and I > 0:
        N = S + I + R

        # Calculate propensities
        a1 = beta * S * I / N        # Transmission
        a2 = gamma * I               # Recovery
        a3 = Lambda                  # Birth
        a4 = mu * S                  # Death of a susceptible
        a5 = mu * I                  # Death of an infected
        a6 = mu * R                  # Death of a recovered
        
        a0 = a1 + a2 + a3 + a4 + a5 + a6

        # Time until next event
        dt = -np.log(np.random.random()) / a0
        t += dt

        # Determine which event occurs
        r = np.random.random() * a0
        if r < a1:
            # Transmission event
            S -= 1
            I += 1
        elif r < a1 + a2:
            # Recovery event
            I -= 1
            R += 1
        elif r < a1 + a2 + a3:
            # Birth event
            S += 1
        elif r < a1 + a2 + a3 + a4:
            # Death of a susceptible
            S -= 1
        elif r < a1 + a2 + a3 + a4 + a5:
            # Death of an infected
            I -= 1
        else:
            # Death of a recovered
            R -= 1

        # Store results
        times.append(t)
        S_values.append(S)
        I_values.append(I)
        R_values.append(R)

    return times, S_values, I_values, R_values


We set the transmission rate to 0.3, recovery rate to 0.1. The initial number of susceptible and infected individuals can vary, the initial number of recovered individual is always 0. We can also set how many times we want to run the Gillespie's Algorithm.

In [32]:
# SIR model without demography
beta = 0.3
gamma = 0.1
max_time = 200
t = np.linspace(0, max_time, 1000)
output = widgets.Output()

def update_plot(S0, I0, num_runs):
    R0 = 0
    fig, ax = plt.subplots(figsize=(10, 6))

    # Solve ODE for deterministic SIR
    solution = odeint(SIR_model, [S0, I0, R0], t, args=(beta, gamma))
    S_det, I_det, R_det = solution.T

    # Run Gillespie algorithm multiple times
    all_S = []
    all_I = []
    all_R = []

    for _ in range(num_runs):
        times, S_values, I_values, R_values = gillespie_algorithm(S0, I0, R0, beta, gamma, max_time)
        all_S.append(np.interp(t, times, S_values))
        all_I.append(np.interp(t, times, I_values))
        all_R.append(np.interp(t, times, R_values))

    # Compute average and standard deviation
    avg_S = np.mean(all_S, axis=0)
    std_S = np.std(all_S, axis=0)
    avg_I = np.mean(all_I, axis=0)
    std_I = np.std(all_I, axis=0)

    # Plotting

    # Deterministic SIR
    ax.plot(t, S_det, label="Susceptible (Deterministic)", linestyle="--")
    ax.plot(t, I_det, label="Infectious (Deterministic)", linestyle="--")

    # Average Stochastic SIR
    ax.plot(t, avg_S, label="Avg Susceptible (Stochastic)")
    ax.fill_between(t, avg_S - std_S, avg_S + std_S, alpha=0.2)
    ax.plot(t, avg_I, label="Avg Infectious (Stochastic)")
    ax.fill_between(t, avg_I - std_I, avg_I + std_I, alpha=0.2)

    ax.set_xlabel("Time")
    ax.set_ylabel("Population")
    ax.set_title(f"Comparison of Deterministic and Stochastic SIR Models ({num_runs} runs)")
    ax.legend(loc="upper right")
    ax.grid(True)
    plt.tight_layout()
    plt.show()

S0_slider = widgets.IntSlider(value=990, min=0, max=1000, step=10, description='S0:')
I0_slider = widgets.IntSlider(value=10, min=0, max=100, step=10, description='I0:')
num_runs_slider = widgets.IntSlider(value=1, min=1, max=1000, description='GA Run:')


widgets.interactive(update_plot, S0=S0_slider, I0=I0_slider, num_runs=num_runs_slider)


interactive(children=(IntSlider(value=990, description='S0:', max=1000, step=10), IntSlider(value=10, descript…

In [31]:
# SIR model with demography
beta = 0.3
gamma = 0.1
Lambda = 5  # birth rate
mu = 0.01  # death rate
max_time = 200
t = np.linspace(0, max_time, 1000)
output = widgets.Output()

def update_plot(S0, I0, num_runs):
    R0 = 0
    with output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(10, 6))
    
        # Solve ODE for deterministic SIR
        solution = odeint(SIR_demography, [S0, I0, R0], t, args=(beta, gamma, Lambda, mu))
        S_det, I_det, R_det = solution.T

        # Run Gillespie algorithm multiple times
        all_S = []
        all_I = []
        all_R = []

        for _ in range(num_runs):
            times, S_values, I_values, R_values = GA_with_demography(S0, I0, R0, beta, gamma, Lambda, mu, max_time)
            all_S.append(np.interp(t, times, S_values))
            all_I.append(np.interp(t, times, I_values))
            all_R.append(np.interp(t, times, R_values))

        # Compute average and standard deviation
        avg_S = np.mean(all_S, axis=0)
        std_S = np.std(all_S, axis=0)
        avg_I = np.mean(all_I, axis=0)
        std_I = np.std(all_I, axis=0)

        # Plotting

        # Deterministic SIR
        ax.plot(t, S_det, label="Susceptible (Deterministic)", linestyle="--")
        ax.plot(t, I_det, label="Infectious (Deterministic)", linestyle="--")

        # Average Stochastic SIR
        ax.plot(t, avg_S, label="Avg Susceptible (Stochastic)")
        ax.fill_between(t, avg_S - std_S, avg_S + std_S, alpha=0.2)
        ax.plot(t, avg_I, label="Avg Infectious (Stochastic)")
        ax.fill_between(t, avg_I - std_I, avg_I + std_I, alpha=0.2)

        ax.set_xlabel("Time")
        ax.set_ylabel("Population")
        ax.set_title(f"Comparison of Deterministic and Stochastic SIR Models with Demography ({num_runs} runs)")
        ax.legend(loc="upper right")
        ax.grid(True)
        plt.tight_layout()
        plt.show()

S0_slider = widgets.IntSlider(value=990, min=0, max=1000, step=10, description='S0:')
I0_slider = widgets.IntSlider(value=10, min=0, max=100, step=10, description='I0:')
num_runs_slider = widgets.IntSlider(value=1, min=1, max=1000, description='GA Run:')


interactive_plot = widgets.interactive(update_plot, S0=S0_slider, I0=I0_slider, num_runs=num_runs_slider)
display(interactive_plot, output)

interactive(children=(IntSlider(value=990, description='S0:', max=1000, step=10), IntSlider(value=10, descript…

Output()